In [98]:
import os
import re
import random
from copy import copy
from collections import Counter, defaultdict

import pandas as pd
import numpy as np

### 1

Реализуйте базовый частотный метод по Шерлоку Холмсу:
- подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
- возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
- расшифруйте их таким частотным методом.


In [17]:
# Чтение и обработка файла из первого пункта
files = {}
path = 'corpora/'
for file in os.listdir(path):
    file_name = file.split('.')[0]
    with open(path + file, 'r') as f:
        files[file_name] = Counter(''.join(re.findall(r'[\w ]+',f.read())).lower())
files.keys()

dict_keys(['WarAndPeaceEng', 'WarAndPeace', 'AnnaKarenina'])

Очень странно, что в русском тескте английские бувы встречаются чаще некоторых русских, например буквы Щ или Ф. Но так как мы хотим расшиврововать русские тексты с помощью русского корпуса и английские с помощью английского, предлагаю оставить только числа, пробелы и символы русского и английского алфавита для русского и английского текста соответственно

In [22]:
files['WarAndPeace'].most_common()

[(' ', 110708),
 ('о', 61282),
 ('а', 45209),
 ('е', 42519),
 ('и', 35838),
 ('н', 35119),
 ('т', 30619),
 ('с', 28128),
 ('л', 27277),
 ('в', 24824),
 ('р', 24570),
 ('к', 19328),
 ('д', 16387),
 ('м', 15940),
 ('у', 15454),
 ('п', 13847),
 ('я', 12477),
 ('г', 11177),
 ('ь', 10498),
 ('ы', 10233),
 ('з', 9602),
 ('б', 9310),
 ('ч', 7349),
 ('й', 6210),
 ('ж', 5460),
 ('ш', 5090),
 ('х', 4600),
 ('e', 4449),
 ('ю', 3495),
 ('ц', 2179),
 ('n', 2066),
 ('s', 2058),
 ('i', 1988),
 ('a', 1887),
 ('r', 1853),
 ('u', 1732),
 ('o', 1724),
 ('t', 1640),
 ('э', 1629),
 ('щ', 1514),
 ('l', 1312),
 ('ф', 1209),
 ('m', 1131),
 ('c', 921),
 ('d', 870),
 ('p', 726),
 ('v', 616),
 ('ё', 431),
 ('h', 416),
 ('é', 367),
 ('b', 317),
 ('q', 292),
 ('f', 290),
 ('ъ', 283),
 ('g', 221),
 ('j', 217),
 ('z', 182),
 ('x', 148),
 ('è', 143),
 ('à', 110),
 ('0', 70),
 ('ê', 61),
 ('y', 60),
 ('1', 53),
 ('k', 41),
 ('w', 37),
 ('2', 26),
 ('ç', 23),
 ('8', 20),
 ('â', 18),
 ('5', 17),
 ('3', 16),
 ('ô', 11),


In [62]:
def get_selected_words(word_list: list, cnt: Counter) -> dict:
    word_set = set(word_list)
    filtred_dict = {}
    for letter in cnt.keys():
        if letter in word_set:
            filtred_dict[letter] = cnt[letter]
            
    return Counter(filtred_dict)

In [110]:
numbers_space = [' '] + [str(i) for i in range(10)]
russian_alf = [chr(i) for i in range(ord('а'), ord('я') + 1)] + numbers_space
english_alf = [chr(i) for i in range(ord('a'), ord('z') + 1)] + numbers_space

In [64]:
corpuses = {}
corpuses['WarAndPeace'] = get_selected_words(russian_alf, files['WarAndPeace'])
corpuses['AnnaKarenina'] = get_selected_words(russian_alf, files['AnnaKarenina'])
corpuses['WarAndPeaceEng'] = get_selected_words(english_alf, files['WarAndPeaceEng'])

Напишем энкодер и декодер

In [51]:
class RandomEncoder:
    def __init__(self, alphabet: list, seed: int = 42):
        self.alpha = alphabet
        self.seed = seed
        self.encoder = self._create_sorted_dict()
        self.decoder = {v : k for k, v in self.encoder.items()}
        
    def _create_sorted_dict(self):
        encoder = {}
        alpha_shuffle = copy(self.alpha)
        random.Random(self.seed).shuffle(alpha_shuffle)
        for l1, l2 in zip(self.alpha, alpha_shuffle):
            encoder[l1] = l2
            
        return encoder
    
    def encode(self, s: str) -> str:
        res = [''] * len(s)
        for idx, letter in enumerate(s.lower()):
            res[idx] = self.encoder.get(letter,letter)
            
        return ''.join(res)
    
    def decode(self, s: str) -> str:
        res = [''] * len(s)
        for idx, letter in enumerate(s.lower()):
            res[idx] = self.decoder.get(letter,letter)
            
        return ''.join(res)

In [52]:
rus_enc = RandomEncoder(russian_alf)
test_s = 'Мама мыла раму'
rus_enc.encode(test_s)
print(f'input : {test_s}\nencode: {rus_enc.encode(test_s)}\ndecode: {rus_enc.decode(rus_enc.encode(test_s))}')

input : Мама мыла раму
encode: ь5ь5ль715лц5ь4
decode: мама мыла раму


In [53]:
rus_enc = RandomEncoder(english_alf)
test_s = 'Hello word, im buisy'
rus_enc.encode(test_s)
print(f'input : {test_s}\nencode: {rus_enc.encode(test_s)}\ndecode: {rus_enc.decode(rus_enc.encode(test_s))}')

input : Hello word, im buisy
encode: ew44l78lqj,7y57uzy9s
decode: hello word, im buisy


In [72]:
# считаем
def frec_decode(base_alf, sentence):
    decoder = {}
    sentence_d = get_selected_words(russian_alf, Counter(sentence.lower()))
    ordered_sentence_letters = list(map(lambda x: x[0], sentence_d.most_common()))
    ordered_base_alf_letters = list(map(lambda x: x[0], base_alf.most_common()))
    for l_s, l_b in zip(ordered_sentence_letters, ordered_base_alf_letters):
        decoder[l_s] = l_b
        
    res = [''] * len(sentence)
    for idx, letter in enumerate(sentence.lower()):
        res[idx] = decoder.get(letter,letter)
            
    return ''.join(res)

Вот несколько примеров работы

In [77]:
alf = english_alf
enc = RandomEncoder(alf)
test_s = 'Hello word, im buisy'
code = enc.encode(test_s)
print(f"""input : {test_s}\
encode: {code}
decode: {enc.decode(code)}
freq_dec: {frec_decode(corpuses['WarAndPeaceEng'], code)}""")

input : Hello word, im buisyencode: ew44l78lqj,7y57uzy9s
decode: hello word, im buisy
freq_dec: eweel tlqj, ya uzyos


In [85]:
alf = english_alf
enc = RandomEncoder(alf)
test_s = 'English texts for beginners to practice reading and comprehension online and for free. Practicing your comprehension of written English will both improve your vocabulary and understanding of grammar and word order. The texts below are designed to help you develop while giving you an instant evaluation of your progress.'
code = enc.encode(test_s)
print(f"""input: {test_s}\n
encode: {code} \n
decode: {enc.decode(code)}\n
WAP: {frec_decode(corpuses['WarAndPeaceEng'], code)}
""")

input: English texts for beginners to practice reading and comprehension online and for free. Practicing your comprehension of written English will both improve your vocabulary and understanding of grammar and word order. The texts below are designed to help you develop while giving you an instant evaluation of your progress.

encode: wk04y9e7gwng97 lq7uw0ykkwq97gl7tqfmgymw7qwfjyk07fkj7ml5tqwewk9ylk7lk4ykw7fkj7 lq7 qww.7tqfmgymyk07slzq7ml5tqwewk9ylk7l 78qyggwk7wk04y9e78y447ulge7y5tqlaw7slzq7almfuz4fqs7fkj7zkjwq9gfkjyk07l 70qf55fq7fkj78lqj7lqjwq.7gew7gwng97uw4l87fqw7jw9y0kwj7gl7ew4t7slz7jwaw4lt78ey4w70yayk07slz7fk7yk9gfkg7waf4zfgylk7l 7slzq7tql0qw99. 

decode: english texts for beginners to practice reading and comprehension online and for free. practicing your comprehension of written english will both improve your vocabulary and understanding of grammar and word order. the texts below are designed to help you develop while giving you an instant evaluation of your progress.

WAP: wktay

In [89]:
alf = russian_alf
enc = RandomEncoder(alf)
test_s = 'Сбербанк увеличил максимальную сумму кредита по ипотеке с господдержкой до 30 млн руб. для Москвы, Московской области, Санкт-Петербурга и Ленинградской области и до 15 млн руб. для других регионов, сообщили в банке.'
code = enc.encode(test_s)
print(f"""input: {test_s}\n
encode: {code} \n
decode: {enc.decode(code)}\n
WAP: {frec_decode(corpuses['WarAndPeace'], code)} \n
AK: {frec_decode(corpuses['AnnaKarenina'], code)}
""")

input: Сбербанк увеличил максимальную сумму кредита по ипотеке с господдержкой до 30 млн руб. для Москвы, Московской области, Санкт-Петербурга и Ленинградской области и до 15 млн руб. для других регионов, сообщили в банке.

encode: 4гт3гйхцынктлд6длыэйц4дэйл2хнаы4нээныц3т5д йышмыдшм тцты4ыъм4шм55т39цм0ы5мыиеыэлхы3нг.ы5лвыэм4цк8,ыэм4цмк4цм0ымглй4 д,ы4йхц -шт т3гн3ъйыдылтхдхъ3й54цм0ымглй4 дыды5мы1пыэлхы3нг.ы5лвы53нъдфы3тъдмхмк,ы4ммгудлдыкыгйхцт. 

decode: сбербанк увеличил максимальную сумму кредита по ипотеке с господдержкой до 30 млн руб. для москвы, московской области, санкт-петербурга и ленинградской области и до 15 млн руб. для других регионов, сообщили в банке.

WAP: евисвлрн дпитазат млнеамлтбрдч едммд нсикаул го агоуини е яоегоккисйноь ко жш мтр сдв. кты моенпх, моенопеноь овтлеуа, елрну-гиуисвдсял а тираряслкеноь овтлеуа а ко юц мтр сдв. кты ксдяаэ сияаороп, еоовщата п влрни. 

AK: авнсвлри дпнтебет млиаемлтчрдз адммд иснкеул ьо еьоунин а яоаьоккнсжиоы ко йш мтр сдв. ктг моаипх,

По итогу выглядит похоже на истину, пробелы всегда расшифровываются верно, если пытаться грузить достаточно длинные тексты, то это можно пытаться читать и получается что-то издали напоминющее нужные слова

### 2

Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведите тестирование аналогично п.1, но при помощи биграмм.


Я сначала реализовал вариант без пробелов(биграммы только на словах делаются), но на полпути понял,
что пробел может быть не пробелом в коде

In [107]:
filter_symbols = lambda alf, x: ''.join(list(filter(lambda x: x in alf, x.lower())))

In [111]:
# Чтение и обработка файла
files = {}
path = 'corpora/'
for file in os.listdir(path):
    file_name = file.split('.')[0]
    if 'Eng' in file_name:
        with open(path + file, 'r') as f:
            files[file_name] = filter_symbols(english_alf, f.read())
    else:
        with open(path + file, 'r') as f:
            files[file_name] = filter_symbols(russian_alf, f.read())
files.keys()

dict_keys(['WarAndPeaceEng', 'WarAndPeace', 'AnnaKarenina'])

In [113]:
files['WarAndPeace'][:101]

'война и мир  самый известный роман льва николаевича толстого как никакое другое произведение писателя'

In [133]:
def get_bigram_counter(corpus: str)->Counter:
    res = defaultdict(int)
    for idx in range(1, len(corpus)):
        pair = corpus[idx - 1] + corpus[idx]
        res[pair] += 1
    return Counter(res)

In [130]:
def frec_bi_decode(base_alf, sentence, filter_alf):
    decoder = {}
    f_sent = filter_symbols(filter_alf, sentence)
    sentence_d = get_bigram_counter(f_sent)
    ordered_sentence_letters = list(map(lambda x: x[0], sentence_d.most_common()))
    ordered_base_alf_letters = list(map(lambda x: x[0], base_alf.most_common()))
    for l_s, l_b in zip(ordered_sentence_letters, ordered_base_alf_letters):
        decoder[l_s] = l_b
        
    res = []
    for idx in range(1, len(f_sent) // 2):
        bigram = f_sent[2*idx: 2*idx + 2]
        res.append(decoder.get(bigram, 'UDT'))
    if len(f_sent) % 2 == 1:
        res.append('UDT')
    
            
    return ''.join(res)

In [134]:
word ='c'
for idx in range(len(word) // 2):
    print(word[2*idx: 2*idx + 2])

In [135]:
test_s = 'Сбербанк увеличил максимальную сумму кредита по ипотеке с господдержкой до 30 млн руб. для Москвы, Московской области, Санкт-Петербурга и Ленинградской области и до 15 млн руб. для других регионов, сообщили в банке.'
frec_bi_decode(get_bigram_counter(files['WarAndPeace']), test_s, russian_alf)

' вне  пр мпорианра гтьк атоге одак рко заз е о уасемжелеойетбыдоми ссткальчт бвоь  деренон пего  к сит сстгоослиа сь  ю х  вруныова ченнтраяди сстгоослиа а камеам бвоь  деренсевынуна лки эорпегослпоейкн нй '

In [138]:
alf = russian_alf
enc = RandomEncoder(alf)
test_s = 'Сбербанк увеличил максимальную сумму кредита по ипотеке с господдержкой до 30 млн руб. для Москвы, Московской области, Санкт-Петербурга и Ленинградской области и до 15 млн руб. для других регионов, сообщили в банке.'
code = enc.encode(test_s)
print(f"""input: {test_s}\n
encode: {code} \n
decode: {enc.decode(code)}\n
WAP: {frec_decode(corpuses['WarAndPeace'], code)} \n
AK: {frec_decode(corpuses['AnnaKarenina'], code)} \n
BiWAP : {frec_bi_decode(get_bigram_counter(files['WarAndPeace']), code, russian_alf)} \n
BiAK : {frec_bi_decode(get_bigram_counter(files['AnnaKarenina']), code, russian_alf)}
""")

input: Сбербанк увеличил максимальную сумму кредита по ипотеке с господдержкой до 30 млн руб. для Москвы, Московской области, Санкт-Петербурга и Ленинградской области и до 15 млн руб. для других регионов, сообщили в банке.

encode: 4гт3гйхцынктлд6длыэйц4дэйл2хнаы4нээныц3т5д йышмыдшм тцты4ыъм4шм55т39цм0ы5мыиеыэлхы3нг.ы5лвыэм4цк8,ыэм4цмк4цм0ымглй4 д,ы4йхц -шт т3гн3ъйыдылтхдхъ3й54цм0ымглй4 дыды5мы1пыэлхы3нг.ы5лвы53нъдфы3тъдмхмк,ы4ммгудлдыкыгйхцт. 

decode: сбербанк увеличил максимальную сумму кредита по ипотеке с господдержкой до 30 млн руб. для москвы, московской области, санкт-петербурга и ленинградской области и до 15 млн руб. для других регионов, сообщили в банке.

WAP: евисвлрн дпитазат млнеамлтбрдч едммд нсикаул го агоуини е яоегоккисйноь ко жш мтр сдв. кты моенпх, моенопеноь овтлеуа, елрну-гиуисвдсял а тираряслкеноь овтлеуа а ко юц мтр сдв. кты ксдяаэ сияаороп, еоовщата п влрни. 

AK: авнсвлри дпнтебет млиаемлтчрдз адммд иснкеул ьо еьоунин а яоаьоккнсжиоы ко йш мтр сдв. ктг моаипх,

Видно, что стало хуже, основная проблема в том, что биграм больше, чем букв, а количество биграм на сопоставимом уровне и получается, на каждую биграмму меньше статистики и результаты хуже. Это можно отлично видеть на примере пробелов, которые очень хорошо определялись при буквенном алгоритме, но существенно хуже при биграммах. Таже по биргаммам есть ряд вопросов:
- что делать с текстами нечетной длины? 
- при декодированни мы смотрели идущую пару и заменяли ее, но возможно имеет смысл сдвигать пары на 1 и качество бы улучшилось?

Получается биграммы более сложный инструмент, который требует больше данных и больше внимания к деталям. Оптимальной была бы смесь биграмм и букв, но это уже другое напрпавление исследования 

### 3

Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуйте и протестируйте его, убедитесь, что результаты улучшились.


In [152]:
text = 'левин думал о евангельском изречении не потому что'

In [215]:
def llh(text, base_alf, filter_alf):
    f_sent = filter_symbols(filter_alf, ''.join(text))
    sentence_d = get_bigram_counter(f_sent)
    llh = np.sum([val * np.log(base_alf.get(key, 1 / len(filter_alf) ** 2)) for key, val in sentence_d.items()])
    
    return llh

In [216]:
llh(text, get_bigram_counter(files['AnnaKarenina']), russian_alf)

435.4853444834954

In [232]:
def change_pair(alf, sentence):
    text = list(sentence)
    permut = np.random.choice(list(alf), 2, replace=False)
    while permut[0] == permut[1]:
        permut = np.random.choice(list(alf), 2, replace=False)
    for iterr in range(len(text)):
        if text[iterr] == permut[0]:
            text[iterr] = permut[1]
        elif text[iterr] == permut[1]:
            text[iterr] = permut[0]
            
    return text

In [239]:
for w1, w2 in zip(change_pair(russian_alf, text), text):
    if w1 != w2:
        print(w1, w2)

1 и
1 и
1 и
1 и


In [173]:
condition = lambda x, y: x > y or np.random.rand() < np.exp(x - y)

In [245]:
def mcmc_decoder(sentence, alf, base_alf, num_iters=10000):
    text = list(sentence.lower())
    decoder_text = 'Не отработал'
    value_likelihood = best_value = llh(text, base_alf, alf)
    for iteration in range(num_iters):
        new_text = change_pair(alf, copy(text))
        new_value_likelihood = llh(new_text, base_alf, alf)
        if condition(new_value_likelihood, value_likelihood):
            text = new_text
            value_likelihood = new_value_likelihood
            if value_likelihood > best_value:
                best_value = value_likelihood
                decoder_text = copy(text)
                
    out_text = ''.join(decoder_text)
    
    return out_text

In [243]:
test_s ="""Ах, не говорите мне про Австрию! Я ничего не понимаю, может быть, но Австрия никогда не хотела и не хочет войны. Она предает нас. Россия одна должна быть спасительницей Европы. Наш благодетель знает свое высокое призвание и будет верен ему. Вот одно, во что я верю. Нашему доброму и чудному государю предстоит величайшая роль в мире, и он так добродетелен и хорош, что Бог не оставит его, и он исполнит свое призвание задавить гидру революции, которая теперь еще ужаснее в лице этого убийцы и злодея. Мы одни должны искупить кровь праведника. На кого нам надеяться, я вас спрашиваю?.. Англия с своим коммерческим духом не поймет и не может понять всю высоту души императора Александра. Она отказалась очистить Мальту. Она хочет видеть, ищет заднюю мысль наших действий. Что они сказали Новосильцеву? Ничего. Они не поняли, они не могли понять самоотвержения нашего императора, который ничего не хочет для себя и все хочет для блага мира. И что они обещали? Ничего. И что обещали, и того не будет! Пруссия уже объявила, что Бонапарте непобедим и что вся Европа ничего не может против него... И я не верю ни в одном слове ни Гарденбергу, ни Гаугвицу. Cette fameuse neutralité prussienne, ce n'est qu'un piège 9. Я верю в одного Бога и в высокую судьбу нашего милого императора. Он спасет Европу!.. — Она вдруг остановилась с улыбкой насмешки над своею горячностью."""

In [250]:
alf = russian_alf
enc = RandomEncoder(alf)
# test_s = 'Сбербанк увеличил максимальную сумму кредита по ипотеке с господдержкой до 30 млн руб. для Москвы, Московской области, Санкт-Петербурга и Ленинградской области и до 15 млн руб. для других регионов, сообщили в банке.'
code = enc.encode(filter_symbols(russian_alf, test_s))
print(f"""input: {filter_symbols(russian_alf, test_s)}\n
encode: {code} \n

MCMC WAP: {mcmc_decoder(code, russian_alf, get_bigram_counter(files['AnnaKarenina']))} \n
""")

input: ах не говорите мне про австрию я ничего не понимаю может быть но австрия никогда не хотела и не хочет войны она предает нас россия одна должна быть спасительницей европы наш благодетель знает свое высокое призвание и будет верен ему вот одно во что я верю нашему доброму и чудному государю предстоит величайшая роль в мире и он так добродетелен и хорош что бог не оставит его и он исполнит свое призвание задавить гидру революции которая теперь еще ужаснее в лице этого убийцы и злодея мы одни должны искупить кровь праведника на кого нам надеяться я вас спрашиваю англия с своим коммерческим духом не поймет и не может понять всю высоту души императора александра она отказалась очистить мальту она хочет видеть ищет заднюю мысль наших действий что они сказали новосильцеву ничего они не поняли они не могли понять самоотвержения нашего императора который ничего не хочет для себя и все хочет для блага мира и что они обещали ничего и что обещали и того не будет пруссия уже объявила что бона

Переводит намного лучше, предыдущих

### 4

←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏
Или это (они одинаковые, второй вариант просто на случай проблем с юникодом):
დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ


In [288]:
massage = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'
alf = list(set(massage))
len(alf)

28

In [273]:
encoder = {k:v for k, v in zip(alf, russian_alf[:len(alf)])}
enc_massage = [encoder.get(i) for i in massage]
''.join(enc_massage)

'тпхжщнмщнжлжитщъцыайхфъмшщжхжщсцгижщъцыайхфъмшщитдпищещвицрцщпццузтъжчщдцицымшщхтрдцщсыцгжийифщпдцыттщнптрцщнмщнптщплтхйхжщсыйнжхфъцщжщсцхегжитщайдпжайхфъмшщуйххщойщсцпхтлъттщгтинтыицтщойлйъжтщдеыпйщбцичщдцътгъцщчщъжгтрцщътщцутзйк'

In [285]:
alf = russian_alf[:-10]
base = get_bigram_counter(filter_symbols(alf, files['WarAndPeace'] + files['AnnaKarenina']))
mcmc_decoder(''.join(enc_massage), alf, base, num_iters=50000)

'если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж'

Текст более менее читаем, это лучший результат